In [1]:
from podium import TabularDataset, Vocab, Field
from podium.vocab import Special
from podium.vectorizers import GloVe
from podium import BucketIterator
from podium.vocab import UNK, PAD, EOS, BOS
import numpy as np

In [2]:
data_path_train_csv = 'dd_dataset/train/train/train.csv'

In [3]:
def lowercase(raw):
    """Lowercases the input string"""
    return raw.lower()

class RemoveBlanks:
    def __call__(self, raw, tokenized):
        """Remove punctuation from tokenized data"""
        return raw, [tok for tok in tokenized if tok not in [' ', "\n", "\t"]]

In [4]:
vocab = Vocab(max_size=10000, min_freq=2, specials=(PAD(),UNK(),BOS(),EOS()))
text = Field('text',
             numericalizer=vocab,
             pretokenize_hooks=[lowercase],
             posttokenize_hooks=[RemoveBlanks()],
             tokenizer='spacy-en_core_web_sm')
fields = {'text': text}

In [5]:
dataset = TabularDataset(data_path_train_csv, format='csv', fields=fields)
dataset.finalize_fields()

In [6]:
vocab = fields['text'].vocab
glove = GloVe()
embeddings = glove.load_vocab(vocab)

In [7]:
print(f"For vocabulary of size: {len(vocab)} loaded embedding matrix of shape: {embeddings.shape}")

For vocabulary of size: 10000 loaded embedding matrix of shape: (10000, 300)


In [8]:
def instance_length(instance):
    _, tokenized = instance.text
    return len(tokenized)

bucket_iterator = BucketIterator(dataset, batch_size=32, bucket_sort_key=instance_length)
vocab

Vocab({specials: ('<PAD>', '<UNK>', '<BOS>', '<EOS>'), eager: False, is_finalized: True, size: 10000})

In [20]:
def get_embeddings(batch, embeddings):
    w2vec = np.zeros((batch.shape[0], batch.shape[1], embeddings.shape[1]))
    for i in range(batch.shape[0]):
        for j in range(batch.shape[1]):
            w2vec[i,j,:] = embeddings[batch[i,j]]
    
    return w2vec

In [21]:
for iterator in bucket_iterator:
    batch = iterator.text
    print(get_embeddings(batch, embeddings).shape)
    break

(32, 4, 300)
